# Project Selection
## Topic Selection
- **Overal idea**: Texture classification using contact microphone data from robotic hand
- **Analysis approaches**:
  - 1D Convolutional Neural Network (CNN) on time-series data
  - 2D CNN on spectrogram
  - Hidden Markov Model (HMM) on time-series
  - HMM on spectrogram
- **Problem statement**: Can CNNs outperform HMM models for texture classification tasks on acoustic contact-sensing data gathered from a robotic hand
- **Relevant resources**:
  - [AU Dataset for Visuo-Haptic Object Recognition for Robots](https://arxiv.org/pdf/2112.13761.pdf)
  - [Publicly available datatset](https://figshare.com/articles/dataset/AU_Dataset_for_Visuo-Haptic_Object_Recognition_for_Robots/14222486)
  - [Another publicly available dataset](https://arxiv.org/pdf/2307.00937.pdf)
- **Precedent literature**:
  - [Journal: identifying pill type based on acoustic data gathered via shaking motion](https://tams.informatik.uni-hamburg.de/people/jonetzko/publications/jonetzko2020multimodal.pdf)
  - [Object classification with transmission acoustic sensing (Prof. Culbertson)](https://arxiv.org/pdf/2308.01600.pdf)
  - [Fabric classification based on acoustic data gathered via sliding motion (dataset avilable)](https://www.frontiersin.org/articles/10.3389/fnbot.2022.808222/full) 

In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.1031, 0.4530, 0.1348],
        [0.9834, 0.8297, 0.9469],
        [0.7635, 0.3847, 0.8169],
        [0.5393, 0.8654, 0.7956],
        [0.5823, 0.3987, 0.1955]])


### Signal Processing Steps
- Hamming window *before* spectral analysis (from *Isolated-word speech recognition using hidden Markov models*)
- Remove ambient spectrum (from *A soft, amorphous skin that can sense and localize textures*)
    - How?
- Scale data by mean and standard deviation and re-sample to account for different time-series lengths (from *Fabric Classification Using a Finger-Shaped Tactile Sensor via Robotic Sliding*)
- Extracting features from the time-series beyond frequency bins (from: *Design of a Biomimetic Tactile Sensor for Material Classification*)
- Spectral subtraction (from: *Evaluating Integration Strategies for Visuo-Haptic Object Recognition*)
    - Cited paper: 
- Rebinning frequency spectrum (from: *Stane: Synthesized surfaces for tactile input*)

### Classification
- CNN + transformer (from: *An Investigation of Multi-feature Extraction and Super-resolution with Fast Microphone Arrays*)

